# 1) Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from geopy.distance import great_circle

# 2) Coleta dos Dados

In [2]:
imputer = IterativeImputer(random_state=42)

## 2.1) Lista de municípios do Sertão

In [3]:
mun_ser = pd.read_excel(r'../Dados/Municipios/lista-1262municipios-semiarido-2017.xlsx', header = 2).iloc[1:]
mun_ser = mun_ser[['Cod IBGE','UF', 'Município']]
mun_ser.columns = ['IBGE7', 'UF', 'NOME']

mun_cord = pd.read_csv(r'../Dados/Municipios/municipios.csv')
mun_cord = mun_cord[['codigo_ibge', 'latitude', 'longitude']]
mun_cord.columns = ['IBGE7', 'LATITUDE', 'LONGITUDE']

mun_ser = mun_ser.merge(mun_cord, on ='IBGE7', how = 'left')
mun_ser

,IBGE7,UF,NOME,LATITUDE,LONGITUDE
0,2700300,AL,ARAPIRACA,-9.75487,-36.6615
1,2700706,AL,BATALHA,-9.67420,-37.1330
2,2700904,AL,BELO MONTE,-9.82272,-37.2770
3,2701209,AL,CACIMBINHAS,-9.40121,-36.9911
4,2701605,AL,CANAPI,-9.11932,-37.5967
...,...,...,...,...,...
1256,2806008,SE,RIBEIRÓPOLIS,-10.53570,-37.4380
1257,2807006,SE,SÃO MIGUEL DO ALEIXO,-10.38470,-37.3836
1258,2807105,SE,SIMÃO DIAS,-10.73870,-37.8097
1259,2807303,SE,TELHA,-10.20640,-36.8818


In [4]:
mun_ser.to_csv('../Dados/Views/municipios_sertao.csv')

## 2.2) Dados de registros administrativos 

In [5]:
def calcular_populacao_total(municipio, municipios_df):
    populacao_total = municipio['POP_TOT']  
    for _, row in municipios_df.iterrows():
        distancia = great_circle((municipio['LATITUDE'], municipio['LONGITUDE']), (row['LATITUDE'], row['LONGITUDE'])).kilometers
        if distancia <= 30:
            populacao_total += row['POP_TOT']
    return populacao_total

metrics = [
    'ANO', 'IBGE7','DIST_EF_PUB','DIST_EM_PUB','TTREVA_EF_PUB','TTREVA_EM_PUB','IDEB_AI',
    'IDEB_AF','DOCSUP_EF_PUB','DOCSUP_EM_PUB','TXNASC7C','TXNBAIXOP','PINTERSAP','PINTERDRSAI',
    'REN_PIBPC_D','PDEFAGUA','PDEFESGOTO','PANALF15','PIND_POS','SNIS_PAGUA','POP_TOT'
]

reg_adm_mun = pd.read_excel(r'../Dados/Registros Administrativos/dados_registros_administrativos_total_2012_2017.xlsx', sheet_name = 'MUNICÍPIO')
reg_adm_mun = reg_adm_mun[reg_adm_mun['IBGE7'].isin(mun_ser['IBGE7'])]
reg_adm_mun = reg_adm_mun[metrics]
reg_adm_mun = reg_adm_mun.dropna(axis = 1, thresh = reg_adm_mun.shape[0]*0.5)

num_metrics = ['ANO', 'DIST_EF_PUB', 'DIST_EM_PUB', 'IDEB_AI',
               'IDEB_AF', 'DOCSUP_EF_PUB', 'DOCSUP_EM_PUB', 'TXNASC7C', 'TXNBAIXOP',
               'PINTERSAP', 'PINTERDRSAI', 'REN_PIBPC_D', 'PDEFAGUA', 'PDEFESGOTO',
               'PANALF15', 'PIND_POS', 'SNIS_PAGUA', 'POP_TOT']


reg_adm_mun[num_metrics] = imputer.fit_transform(reg_adm_mun[num_metrics])
reg_adm_mun = reg_adm_mun.drop(columns='ANO')
reg_adm_mun = reg_adm_mun.groupby(['IBGE7']).mean().reset_index()

cidades = mun_ser.copy()

cidades = cidades.merge(reg_adm_mun, on = 'IBGE7', how = 'inner')

cidades['POP_TOT_30KM'] = 0

for index, municipio in cidades.iterrows():
    populacao_total_30km = calcular_populacao_total(municipio, cidades)
    cidades.at[index, 'POP_TOT_30KM'] = populacao_total_30km

cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades

,IBGE7,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,TXNASC7C,TXNBAIXOP,PINTERSAP,PINTERDRSAI,REN_PIBPC_D,PDEFAGUA,PDEFESGOTO,PANALF15,PIND_POS,SNIS_PAGUA,POP_TOT,POP_TOT_30KM
0,2700300,28.006125,42.522954,4.658296,3.683754,54.40,56.96,60.158262,10.090574,18.636148,6.815611,11.476737,25.620092,73.980255,16.520228,41.005022,97.486,230975.6,582087.0
1,2700706,30.831978,36.687844,4.495362,3.590319,12.62,41.94,55.029615,7.258357,30.860307,12.150951,5.413462,45.799184,83.940028,18.260767,67.269176,88.194,18487.6,120483.6
2,2700904,34.847867,52.813493,4.083426,3.171860,27.12,31.84,53.794486,8.063568,20.308137,6.723170,6.975301,59.319507,82.456248,23.811118,43.895632,91.050,6774.4,114803.4
3,2701209,38.654232,43.228540,4.104964,3.317861,49.04,45.22,52.443551,6.698345,17.231791,2.046380,5.351852,76.988586,62.767590,22.908079,52.907744,32.604,10815.6,141797.4
4,2701605,31.898935,36.839351,4.667102,3.633118,20.84,32.34,52.780364,8.178384,12.902533,1.653252,4.041184,73.038977,56.570433,24.698569,39.389925,94.054,17958.4,148162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,2806008,32.467580,40.276968,4.230984,3.565478,64.68,77.28,73.853258,6.865284,8.830588,0.186191,8.988814,27.970823,37.965023,10.780480,48.524164,96.058,18359.0,100748.8
1257,2807006,32.107015,47.482277,3.915938,3.184283,62.08,66.38,55.382784,8.533114,16.568992,1.666673,6.807009,34.856450,47.964368,14.682419,51.531346,95.810,3901.6,127390.8
1258,2807105,35.640213,43.864958,4.286000,3.467918,61.38,82.96,54.129258,7.443652,20.306567,0.782715,8.691907,46.451116,41.689643,15.650402,61.263952,97.866,40522.2,120552.4
1259,2807303,29.921124,45.640696,4.361086,3.524588,45.66,71.74,59.483911,7.337448,12.372443,0.612512,6.842277,18.654712,25.516423,13.314568,62.733835,98.560,3169.6,81132.8


In [6]:
cidades.to_csv('../Dados/Views/registros_administrativos.csv')

## 2.3) Dados meteorológicos 

In [7]:
def calcular_distancia(cidade, estacao):
    cidade_coord = (cidade['LATITUDE'], cidade['LONGITUDE'])
    estacao_coord = (estacao['LATITUDE'], estacao['LONGITUDE'])
    return great_circle(cidade_coord, estacao_coord).kilometers

ufs = list(set(mun_ser['UF']))
est_met = pd.DataFrame()

num_columns = [
    'LATITUDE', 'LONGITUDE', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
]

for year in [2020, 2021, 2022]:
    
    file_path = f'../Dados/INMET/INMET_{year}.csv'
    file = pd.read_csv(file_path)
    file = file[['Data', 'UF', 'ESTACAO', 'LATITUDE', 'LONGITUDE', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"]]
    file = file[file['UF'].isin(ufs)]
    file = file.dropna(axis = 0, thresh = 6)
    file[num_columns] = imputer.fit_transform(file[num_columns])
    file = file.groupby(['Data', 'UF', 'ESTACAO','LATITUDE', 'LONGITUDE']).agg({'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'sum', "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": 'mean'}).reset_index()
    est_met = pd.concat([est_met, file], ignore_index=True)

est_met = est_met.drop(columns = 'Data')
est_met = est_met.groupby(['ESTACAO', 'UF']).mean().reset_index()
est_met.columns = ['ESTACAO', 'UF', 'LATITUDE', 'LONGITUDE', 'PREC_MED', 'TEMP_MED']

cidades = mun_ser.copy()

for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in est_met.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    estacao_mais_proxima = est_met.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'PREC_MED'] = estacao_mais_proxima['PREC_MED']
    cidades.at[index_cidade, 'TEMP_MED'] = estacao_mais_proxima['TEMP_MED']
    
cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades

,IBGE7,PREC_MED,TEMP_MED
0,2700300,2.263969,26.327873
1,2700706,1.953488,27.754398
2,2700904,1.953488,27.754398
3,2701209,2.296316,25.157018
4,2701605,1.516842,27.046445
...,...,...,...
1256,2806008,0.984831,26.969830
1257,2807006,1.894041,25.452166
1258,2807105,2.499337,24.440877
1259,2807303,4.713677,26.562289


In [8]:
cidades.to_csv('../Dados/Views/dados_meteorologicos.csv')

## 2.4) Dados de recursos hídricos

In [9]:
atl_irr = pd.read_excel(r'../Dados/ANA/Atlas Irrigacao - Area Atual e Potencial.xlsx',header = 6).iloc[6:]
atl_irr = atl_irr[['Código','Área Total Irrigada', 'AAI - Potencial Total', 'AAI - Potencial Efetivo']]
atl_irr = atl_irr[atl_irr['Código'].isin(mun_ser['IBGE7'])]
atl_irr.columns = ['IBGE7','AREA_IRRIGADA_TOT', 'AREA_IRRIGADA_POT', 'AREA_IRRIGADA_POT_E']
atl_irr

,IBGE7,AREA_IRRIGADA_TOT,AREA_IRRIGADA_POT,AREA_IRRIGADA_POT_E
465,2100907,56.000000,829.401141,3.140248
653,2112209,666.262426,4269.334204,0.000000
667,2200053,96.000000,0.000000,0.000000
670,2200251,0.000000,1240.907963,0.000000
671,2200277,2.000000,0.000000,0.000000
...,...,...,...,...
3076,3170651,353.204987,1106.772079,142.399238
3079,3170800,4133.645939,37599.860831,1881.799953
3080,3170909,197.000000,400.497303,1.268556
3082,3171030,4822.626564,295.487156,3.395124


In [10]:
atl_irr.to_csv('../Dados/Views/recursos_hidricos.csv')

## 2.3) Dados do censo

In [11]:
censo = pd.read_excel(r'../Dados/Censo/Censo_municipal_estadual_nacional.xlsx', sheet_name = 'MUN 91-00-10')
censo = censo[(censo['Codmun7'].isin(mun_ser['IBGE7']))&(censo['ANO']==2010)]
censo_colunas = ['Codmun7', 'IDHM', 'T_BANAGUA','T_DENS','T_LIXO','T_LUZ','AGUA_ESGOTO',
                 'PAREDE', 'PIND', 'PINDCRI', 'T_DES18M', 'T_FBBAS', 'T_FBFUND', 'T_FBMED', 
                 'T_FBPRE', 'T_FBSUPER', 'T_ATRASO_1_BASICO', 'T_ATRASO_1_FUND', 'T_ATRASO_1_MED',
                 'T_ATRASO_2_BASICO', 'T_ATRASO_2_FUND', 'T_ATRASO_2_MED', 'T_ANALF11A14',
                 'T_ANALF15A17', 'T_ANALF15M', 'T_ANALF18A24', 'T_ANALF18M', 'T_ANALF25A29',
                 'T_ANALF25M']

censo = censo[censo_colunas]
censo.columns = censo_colunas
censo




,Codmun7,IDHM,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,PIND,PINDCRI,...,T_ATRASO_2_BASICO,T_ATRASO_2_FUND,T_ATRASO_2_MED,T_ANALF11A14,T_ANALF15A17,T_ANALF15M,T_ANALF18A24,T_ANALF18M,T_ANALF25A29,T_ANALF25M
11594,2100907,0.521,31.94,42.14,71.79,95.54,23.23,33.06,40.55,49.69,...,32.33,25.21,12.84,11.16,9.96,33.70,13.71,36.20,22.16,42.60
11782,2112209,0.649,76.38,39.60,77.55,98.66,7.39,17.06,9.62,15.38,...,24.72,19.87,10.84,6.32,3.28,17.26,4.16,18.49,7.86,22.40
11796,2200053,0.528,19.28,36.52,99.22,98.99,23.32,0.53,32.12,46.41,...,28.15,20.35,15.65,7.49,4.62,30.41,6.79,32.67,14.49,39.08
11799,2200251,0.531,48.55,24.04,97.24,86.10,10.77,0.21,28.91,41.95,...,38.92,35.54,6.81,15.70,11.72,44.40,14.57,46.90,27.83,53.81
11800,2200277,0.585,72.12,34.60,97.43,98.34,3.89,0.00,22.97,32.09,...,30.75,26.90,3.28,4.73,5.70,35.26,6.92,37.95,17.77,45.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14205,3170651,0.634,75.25,18.35,88.58,96.62,16.25,0.16,15.35,21.42,...,13.82,9.77,3.17,2.81,1.82,25.27,3.03,27.77,8.74,34.48
14208,3170800,0.666,85.76,26.52,97.24,99.13,9.12,0.63,4.81,6.75,...,20.40,17.76,9.39,1.85,1.15,12.00,1.94,12.97,3.02,15.69
14209,3170909,0.594,66.33,37.32,67.62,98.34,17.92,0.38,22.81,30.83,...,25.06,20.02,18.77,2.58,2.42,24.45,4.64,27.06,8.40,33.46
14211,3171030,0.584,59.85,40.46,95.64,95.35,16.28,4.68,14.73,18.72,...,26.21,22.74,9.13,6.76,4.15,26.85,6.22,30.03,12.69,37.53


In [12]:
censo.to_csv('../Dados/Views/censo.csv')

## 2.4) Qualidade da água

In [13]:
ql_agua = pd.read_csv(r'../Dados/ANA/Indicadores de Qualidade da Agua.csv')
ql_agua = ql_agua[['CDESTACAO','CORPODAGUA', 'SGUF','LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']]
ql_agua = ql_agua[ql_agua['SGUF'].isin(ufs)]
ql_agua = ql_agua.dropna(axis = 0, thresh = 3)

num_columns = ['LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']

ql_agua[num_columns] = imputer.fit_transform(ql_agua[num_columns])
ql_agua['QUAL_MED_AGUA'] = (ql_agua['MED_2021'] + ql_agua['MED_2020'])/2
ql_agua = ql_agua.reset_index().drop(columns='index')
ql_agua

cidades = mun_ser.copy()
dist_corpo_agua_mais_prox = []

# Loop para calcular a distância e encontrar a estação mais próxima para cada cidade
for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in ql_agua.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    dist_corpo_agua_mais_prox.append(min(distancias))
    estacao_mais_proxima = ql_agua.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'QUAL_MED_AGUA'] = estacao_mais_proxima['QUAL_MED_AGUA']

cidades['DIST_CORPO_AGUA'] = dist_corpo_agua_mais_prox
cidades = cidades.drop(columns = ['UF', 'NOME', 'LATITUDE', 'LONGITUDE'])
cidades

c:\Users\caios\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,IBGE7,QUAL_MED_AGUA,DIST_CORPO_AGUA
0,2700300,62.439167,42.035186
1,2700706,65.300843,24.642419
2,2700904,65.300843,3.297594
3,2701209,65.300843,58.541833
4,2701605,66.893107,68.942666
...,...,...,...
1256,2806008,64.911294,1.388835
1257,2807006,64.911294,16.508752
1258,2807105,65.088090,2.685846
1259,2807303,64.553680,4.298665


In [14]:
cidades.to_csv('../Dados/Views/qualidade_da_agua.csv')

## 2.5) Dados de transporte

In [15]:
trans = pd.read_excel('../Dados/Transportes/transport_cost.xlsx')
trans = trans[['codigo_ibge','transportation_cost']].groupby('codigo_ibge').min().reset_index()
trans.columns = ['IBGE7', 'TRANSPORT_COST']
trans

,IBGE7,TRANSPORT_COST
0,1400100,40478.947612
1,1400282,40326.183579
2,1501402,33091.358691
3,1501600,32666.086719
4,1504802,35154.250138
...,...,...
1380,5213806,25385.396775
1381,5214051,27720.752548
1382,5217203,27268.044018
1383,5219803,26161.653604


In [16]:
trans.to_csv('../Dados/Views/custo_de_transporte.csv')

## 2.6) Tabela final

In [21]:
mun_ser = pd.read_csv('../Dados/Views/municipios_sertao.csv').drop(columns = 'Unnamed: 0')
reg_adm_mun = pd.read_csv('../Dados/Views/registros_administrativos.csv').drop(columns = 'Unnamed: 0')
met = pd.read_csv('../Dados/Views/dados_meteorologicos.csv').drop(columns = 'Unnamed: 0')
rec_h = pd.read_csv('../Dados/Views/recursos_hidricos.csv').drop(columns = 'Unnamed: 0')
censo = pd.read_csv('../Dados/Views/censo.csv').drop(columns = 'Unnamed: 0')
ql_agua = pd.read_csv('../Dados/Views/qualidade_da_agua.csv').drop(columns = 'Unnamed: 0')
trans = pd.read_csv('../Dados/Views/custo_de_transporte.csv').drop(columns = 'Unnamed: 0')

tabela_final = mun_ser.copy()
tabela_final = tabela_final.merge(reg_adm_mun, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(met, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(rec_h, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(censo, right_on='Codmun7', left_on= 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(ql_agua, on = 'IBGE7', how = 'left')
tabela_final = tabela_final.merge(trans, on = 'IBGE7', how = 'left')

num_metrics = [column for column in tabela_final.columns if column not in ['IBGE7', 'UF', 'NOME', 'LATITUDE', 'LONGITUDE']]
tabela_final[num_metrics] = imputer.fit_transform(tabela_final[num_metrics])
tabela_final.drop(columns="Codmun7", inplace=True)
tabela_final.to_csv('../Dados/Tabela_final/tabela_final.csv', index=False)